# **CellPhoneDB Final Analysis**

In [1]:
import os 
import pandas as pd 
import glob
import sys
import ktplotspy as kpy
import matplotlib.pyplot as plt
from matplotlib import cm
import scanpy as sc
import math
import numpy as np
import anndata as ad
import seaborn as sns

## **Analysis**

### **Install the database**

In [2]:
os.chdir('C:\\Users\\emine\\OneDrive\\Desktop\\HillmanLab\\TsankovaLab\\dataset')
# -- Version of the database
cpdb_version = 'v5.0.0'

# -- Path where the input files to generate the database are located
cpdb_target_dir = os.path.join(cpdb_version)

from cellphonedb.utils import db_utils

db_utils.download_database(cpdb_target_dir, cpdb_version)

Downloaded cellphonedb.zip into v5.0.0
Downloaded complex_input.csv into v5.0.0
Downloaded gene_input.csv into v5.0.0
Downloaded interaction_input.csv into v5.0.0
Downloaded protein_input.csv into v5.0.0
Downloaded uniprot_synonyms.tsv into v5.0.0\sources
Downloaded transcription_factor_input.csv into v5.0.0\sources


### **Data paths**

In [3]:
cpdb_file_path = 'v5.0.0/cellphonedb.zip'
meta_file_path = 'GBM_meta_core_100.tsv'
counts_file_path = 'GBM_mtx_core_100'
out_path = 'results/method2_withScore_core_100'

In [4]:
cpdb_file_path = 'v5.0.0/cellphonedb.zip'
meta_file_path = 'GBM_meta_edge_100.tsv'
counts_file_path = 'GBM_mtx_edge_100'
out_path = 'results/method2_withScore_edge_100'

In [ ]:
from cellphonedb.src.core.methods import cpdb_statistical_analysis_method

cpdb_results = cpdb_statistical_analysis_method.call(
    cpdb_file_path = cpdb_file_path,                 # mandatory: CellphoneDB database zip file.
    meta_file_path = meta_file_path,                 # mandatory: tsv file defining barcodes to cell label.
    counts_file_path = counts_file_path,             # mandatory: normalized count matrix - a path to the counts file, or an in-memory AnnData object
    counts_data = 'hgnc_symbol',                     # defines the gene annotation in counts matrix.
    score_interactions = True,                       # optional: whether to score interactions or not. 
    iterations = 1000,                               # denotes the number of shufflings performed in the analysis.
    threshold = 0.1,                                 # defines the min % of cells expressing a gene for this to be employed in the analysis.
    threads = 5,                                     # number of threads to use in the analysis.
    debug_seed = 42,                                 # debug randome seed. To disable >=0.
    result_precision = 3,                            # Sets the rounding for the mean values in significan_means.
    pvalue = 0.05,                                   # P-value threshold to employ for significance.
    subsampling = False,                             # To enable subsampling the data (geometri sketching).
    subsampling_log = False,                         # (mandatory) enable subsampling log1p for non log-transformed data inputs.
    subsampling_num_pc = 100,                        # Number of componets to subsample via geometric skectching (dafault: 100).
    subsampling_num_cells = 1000,                    # Number of cells to subsample (integer) (default: 1/3 of the dataset).
    separator = '|',                                 # Sets the string to employ to separate cells in the results dataframes "cellA|CellB".
    debug = False,                                   # Saves all intermediate tables employed during the analysis in pkl format.
    output_path = out_path,                          # Path to save results.
    output_suffix = None                             # Replaces the timestamp in the output files by a user defined string in the  (default: None).
    )

## **Visualization**

### **Load output files**

In [5]:
#adata all
os.chdir('C:\\Users\\dataset')
adata = ad.read_h5ad('seurat_all.h5ad')
anno = pd.read_csv('GBM_meta_all.tsv',sep = '\t')
adata.obs = anno

#for edge
os.chdir('C:\\Users\\results\\method2_withScore_edge_100')
means_edge = pd.read_csv('means.txt',sep = '\t')
pvalues_edge = pd.read_csv('pvalues.txt', sep = '\t')

#for core
#read files
os.chdir('C:\\Users\\results\\method2_withScore_core_100')
means_core = pd.read_csv('means.txt',sep = '\t')
pvalues_core = pd.read_csv('pvalues.txt', sep = '\t')

C:\Users\emine\anaconda3\envs\cpdb\lib\site-packages\anndata\compat\__init__.py:229: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.


### **Heatmaps**

In [ ]:
#for core
core_cells = ['GBM_Core', 'Oligo_Core', 'Myeloid_Core', 'BVC_Core']

core_heatmap = np.zeros((4,4))
row = -1
for i in core_cells:
    col = -1
    row = row + 1
    for j in core_cells:
        col = col + 1
        interaction1 = i + "|" + j
        col_sig1 = pvalues_core[pvalues_core[interaction1] < 0.05].reset_index(drop=True)
        interaction2 = j + "|" + i
        col_sig2 = pvalues_core[pvalues_core[interaction2] < 0.05].reset_index(drop=True)
        # Add 'column2' to the end of 'column1'
        combined_column = pd.concat([col_sig1['interacting_pair'], col_sig2['interacting_pair']], ignore_index=True)
        # Get unique elements from the combined column
        unique_elements = combined_column.unique()
        count = len(unique_elements)
        core_heatmap[row,col] = count
        core_heatmap[col,row] = count
print(np.max(core_heatmap))        

# Create a heatmap
plt.figure(figsize=(10, 10))  # Set figure size (optional)
sns.heatmap(core_heatmap, annot=False, cmap='viridis', vmin = 0, vmax = 268 , yticklabels = core_cells, xticklabels = core_cells, cbar=True)  # annot=True adds the data values in each cell
plt.title("Number of significant interaction pairs in the core", fontsize=16, fontweight='bold')
plt.savefig("number_of_significant_interactions_in_the_core.png", bbox_inches='tight')
plt.savefig("number_of_significant_interactions_in_the_core.pdf", format="pdf", bbox_inches="tight")
plt.show()

In [ ]:
#for edge
edge_cells = ['GBM_Margin', 'In-neuron_Margin', 'Ex-neuron_Margin', 'abn-neuron_Margin', 'Astrocyte_Margin', 
              'Oligo_Margin', 'OPC_Margin', 'OL-MG_Margin','T-cell_Margin', 'Myeloid_Margin', 'BVC_Margin']


edge_heatmap = np.zeros((11,11))
row = -1
for i in edge_cells:
    col = -1
    row = row + 1
    for j in edge_cells:
        col = col + 1
        interaction1 = i + "|" + j
        col_sig1 = pvalues_edge[pvalues_edge[interaction1] < 0.05].reset_index(drop=True)
        interaction2 = j + "|" + i
        col_sig2 = pvalues_edge[pvalues_edge[interaction2] < 0.05].reset_index(drop=True)
        # Add 'column2' to the end of 'column1'
        combined_column = pd.concat([col_sig1['interacting_pair'], col_sig2['interacting_pair']], ignore_index=True)
        # Get unique elements from the combined column
        unique_elements = combined_column.unique()
        count = len(unique_elements)
        edge_heatmap[row,col] = count
        edge_heatmap[col,row] = count
print(np.max(edge_heatmap))          
# Create a heatmap
plt.figure(figsize=(10, 10))  # Set figure size (optional)
sns.heatmap(edge_heatmap, annot=False, cmap='viridis', vmin = 0, vmax = 268, yticklabels = edge_cells, xticklabels = edge_cells, cbar=True)  # annot=True adds the data values in each cell
plt.title("Number of significant interaction pairs in the edge", fontsize=16, fontweight='bold')
plt.savefig("number_of_significant_interactions_in_the_edge.png", bbox_inches='tight')
plt.savefig("number_of_significant_interactions_in_the_edge.pdf", format="pdf", bbox_inches="tight")
plt.show()

### **Dot plots**

### **Pick interesting interactions**

In [ ]:
#interactions for dotplot
egfr_interactions = ["NRG2_ERBB3", "BTC_EGFR", "EREG_EGFR", "HBEGF_ERBB4", "NRG2_ERBB4", 
                     "EPGN_EGFR", "EGF_EGFR", "AREG_EGFR", "NRG1_ERBB4", "NRG1_integrin_a6b4_complex", 
                     "NRG4_ERBB4", "BTC_ERBB4", "NRG1_ERBB3", "HBEGF_ERBB2", "HBEGF_EGFR", 
                     "BTC_ERBB3", "EREG_ERBB4", "NRG3_ERBB4", "TGFA_EGFR"]

NOTCH_interactions = [
    "CNTN1_NOTCH1", "CNTN1_NOTCH2", "CNTN1_NRCAM", "DLL1_NOTCH1", "DLL1_NOTCH2", "DLL1_NOTCH3", 
    "DLL1_NOTCH4", "DLL3_NOTCH1", "DLL3_NOTCH2", "DLL3_NOTCH3", "DLL3_NOTCH4", "JAG1_NOTCH1", 
    "JAG1_NOTCH2", "JAG1_NOTCH3", "JAG1_NOTCH4", "JAG2_NOTCH1", "JAG2_NOTCH2", "JAG2_NOTCH3", 
    "JAG2_NOTCH4"]

WNT_interactions = ["WNT2B_WIF1", "WNT2B_FRZB", "WNT5A_FRZB", "WNT5B_FRZB", "WNT5A_ROR1", 
    "WNT5A_ROR2","WNT5A_WIF1", "WNT5B_WIF1", "WNT7B_WIF1", "WNT2B_FZD3_LRP5", "WNT2B_FZD3_LRP6", 
    "WNT2B_FZD4_LRP5", "WNT2B_FZD4_LRP6", "WNT2B_FZD6_LRP5", "WNT2B_FZD6_LRP6", "WNT5A_FZD3_LRP5", 
    "WNT5A_FZD3_LRP6", "WNT5A_FZD4_LRP5", "WNT5A_FZD4_LRP6", "WNT5A_FZD6_LRP5", "WNT5A_FZD6_LRP6", 
    "WNT5B_FZD3_LRP5", "WNT5B_FZD3_LRP6", "WNT5B_FZD4_LRP5", "WNT5B_FZD4_LRP6", "WNT5B_FZD6_LRP5", 
    "WNT5B_FZD6_LRP6", "WNT7B_FZD3_LRP5", "WNT7B_FZD4_LRP6", "WNT7B_FZD6_LRP5", "WNT7B_FZD6_LRP6"]

interesting_interactions1 = ["PTN_PTPRZ1", "MDK_PTPRZ1", "LGALS3_MERTK", "HLA-E_VSIR", "HLA-E_CD94:NKG2C", 
    "FBN1_integrin_a5b1_complex", "FN1_integrin_a8b1_complex", "TNC_integrin_a8b1_complex", 
    "GJA1_GJA1", "TNC_integrin_a9b1_complex", "COL3A1_integrin_a10b1_complex", 
    "COL4A1_integrin_a11b1_complex", "COL5A2_integrin_a10b1_complex", 
    "COL5A3_integrin_a11b1_complex", "COL6A1_integrin_a10b1_complex", 
    "COL7A1_integrin_a10b1_complex", "COL7A1_integrin_a11b1_complex", 
    "COL9A3_integrin_a11b1_complex"]

interesting_interactions2 = ["integrin_aVb1_complex_ADGRE5", 
    "FN1_integrin_a10b1_complex", "GABA_byGAD1_and_SLC6A1_GABBR1", "GDF11_TGFR_AVR2A", 
    "Glutamate_byGLS_and_SLC1A1_Glutamate_Kainate_1_5_complex", 
    "Glutamate_byGLS_and_SLC1A1_Glutamate_Kainate_2_5_complex", 
    "Glutamate_byGLS_and_SLC1A1_Glutamate_Kainate_3_5_complex", 
    "Glutamate_byGLS_and_SLC1A1_Glutamate_NMDA_1_2D_complex", 
    "Glutamate_byGLS_and_SLC17A7_GRM3", "Glutamate_byGLS_and_SLC1A2_GRM3", 
    "GDF11_TGFR_AVR2B", "MDK_ALK", "TENM1_ADGRL1", "NRG3_ERBB4", "JAG1_VASN", 
    "JAG2_VASN", "IGFBP3_TMEM219"]  

all_cells = edge_cells + core_cells 

### **Plot for interactions involving GBM cells**

In [ ]:
# Replace the genes variable in the plot_cpdb to get outputs for specific list of interactions

all_cells = ["GBM_Margin|In-neuron_Margin|Ex-neuron_Margin|abn-neuron_Margin|Astrocyte_Margin|Oligo_Margin|OPC_Margin|OL-MG_Margin|T-cell_Margin|Myeloid_Margin|BVC_Margin|GBM_Core|Oligo_Core|Myeloid_Core|BVC_Core"]

pvalues_core_sub =  pvalues_core[pvalues_core['interacting_pair'].isin(egfr_interactions)].copy().reset_index(drop=True)
pvalues_edge_sub = pvalues_edge[pvalues_edge['interacting_pair'].isin(egfr_interactions)].copy().reset_index(drop=True)

means_core_sub = means_core[means_core['interacting_pair'].isin(egfr_interactions)].copy().reset_index(drop=True)
means_edge_sub = means_edge[means_edge['interacting_pair'].isin(egfr_interactions)].copy().reset_index(drop=True)

p_concat = pd.concat([pvalues_edge_sub.iloc[:,:13],pvalues_edge_sub.filter(like="GBM",axis=1),pvalues_core_sub.filter(like="GBM",axis=1)],axis =1)
    
# Concatenating the relevant columns
m_concat = pd.concat([means_edge_sub.iloc[:, :13], means_edge_sub.filter(like="GBM",axis=1), means_core_sub.filter(like="GBM",axis=1)], axis=1)

#dotplot with all cells
# cell names are all_cells
p = kpy.plot_cpdb(
        adata = adata,
        cell_type1 = all_cells,
        cell_type2 = all_cells,
        means = m_concat,
        pvals = p_concat,
        celltype_key = "CellType.CI",
        genes = egfr_interactions,
        figsize = (15, 15),
        title = "EGFR Interactions",
        max_size = 10,
        highlight_size = 1.5,
        degs_analysis = False,
        standard_scale = True,
        interaction_scores = None,
        scale_alpha_by_interaction_scores = False,
        keep_significant_only = False
    )
p.save("egfr_only_GBM_dotplot_scaled.png", bbox_inches='tight')
p.save("egfr_only_GBM_dotplot_scaled.pdf", format = "pdf", bbox_inches='tight')